In [1]:
!pip3 install rouge

In [2]:
import pandas as pd

file_path = 'responses.csv'
data = pd.read_csv(file_path)

In [11]:
references = data['Ground Truth Answer'].astype(str).apply(lambda x: x.split())
scaled_dot_product_outputs = data['Output (Scaled dot-product)'].astype(str).apply(lambda x: x.split())
multiplicative_outputs = data['Output (Multiplicative)'].astype(str).apply(lambda x: x.split())
double_multiplicative_outputs = data['Output (Double Multiplicative)'].astype(str).apply(lambda x: x.split())

In [12]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Smoothing function
smoothing = SmoothingFunction().method1

# Calculate BLEU scores
bleu_scores_scaled_dot_product = [
    sentence_bleu(ref, out, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing) for ref, out in zip(references, scaled_dot_product_outputs)
]
bleu_scores_multiplicative = [
    sentence_bleu(ref, out, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing) for ref, out in zip(references, multiplicative_outputs)
]
bleu_scores_double_multiplicative = [
    sentence_bleu(ref, out, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing) for ref, out in zip(references, double_multiplicative_outputs)
]

# Compute average BLEU score
average_bleu_scaled_dot_product = sum(bleu_scores_scaled_dot_product) / len(bleu_scores_scaled_dot_product)
average_bleu_multiplicative = sum(bleu_scores_multiplicative) / len(bleu_scores_multiplicative)
average_bleu_double_multiplicative = sum(bleu_scores_double_multiplicative) / len(bleu_scores_double_multiplicative)

average_bleu_scaled_dot_product, average_bleu_multiplicative, average_bleu_double_multiplicative

(0.002635638311331966, 0.001393110925394761, 0.00039649573385989784)

In [5]:
scaled_dot_product_outputs

0                                                  [41]
1                       [the, coliseum, of, the, state]
2                                       [mike, woodson]
3                                               [three]
4     [6, ft, 8, in, -, foot, -, foot, -, foot, -, 1...
5     [larry, bird, of, the, nba, all, -, of, the, n...
6     [michael, jordan., 4., 6., 6., 4., 4., 4., 4.,...
7           [2004, of, the, nba., of, the, nba, finals]
8                                       [mike, woodson]
9     [boston, celtics, -, of, the, ', s, basketball...
10                                              [white]
Name: Output (Scaled dot-product), dtype: object

In [6]:
from rouge import Rouge

# Initialize ROUGE evaluator
rouge = Rouge()

rogue_scores_scaled = []
rouge_scores_multiplicative = []
rouge_scores_double_multiplicative = []

for ref, scaled_dot_product_outs, multi_out, double_out in zip(references, scaled_dot_product_outputs, multiplicative_outputs, double_multiplicative_outputs):
    ref_text = " ".join(ref)
    scaled_text = " ".join(scaled_dot_product_outs)
    multi_text = " ".join(multi_out)
    double_text = " ".join(double_out)
    
    # ROUGE scores
    rogue_scores_scaled.append(rouge.get_scores(scaled_text, ref_text, avg=True))
    rouge_scores_multiplicative.append(rouge.get_scores(multi_text, ref_text, avg=True))
    rouge_scores_double_multiplicative.append(rouge.get_scores(double_text, ref_text, avg=True))

average_rouge_scaled = {
    key: sum(score[key]['f'] for score in rogue_scores_scaled) / len(rogue_scores_scaled)
    for key in rogue_scores_scaled[0]
}
average_rouge_multiplicative = {
    key: sum(score[key]['f'] for score in rouge_scores_multiplicative) / len(rouge_scores_multiplicative)
    for key in rouge_scores_multiplicative[0]
}
average_rouge_double_multiplicative = {
    key: sum(score[key]['f'] for score in rouge_scores_double_multiplicative) / len(rouge_scores_double_multiplicative)
    for key in rouge_scores_double_multiplicative[0]
}

print(average_rouge_scaled, average_rouge_multiplicative, average_rouge_double_multiplicative)

{'rouge-1': 0.28630460254850965, 'rouge-2': 0.04885215725767502, 'rouge-l': 0.28630460254850965} {'rouge-1': 0.17373737254433222, 'rouge-2': 0.03655527562174877, 'rouge-l': 0.17373737254433222} {'rouge-1': 0.17146464506481832, 'rouge-2': 0.03324675277975511, 'rouge-l': 0.17146464506481832}
